In [9]:
from sklearn.metrics.pairwise import cosine_similarity



In [14]:
import streamlit as st
import pickle
import re
import PyPDF2

# Load files
model = pickle.load(open("resume_model.pkl", "rb"))
tfidf = pickle.load(open("tfidf_vectorizer.pkl", "rb"))
job_role_skills = pickle.load(open("job_role_skills.pkl", "rb"))

le = pickle.load(open("label_encoder.pkl", "rb"))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

def extract_text_from_pdf(file):
    reader = PyPDF2.PdfReader(file)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

st.set_page_config(page_title="Resume Screening System", page_icon="📄")
st.title("📄 Resume Screening System")
st.write("Upload your resume to predict the suitable job role.")

uploaded_file = st.file_uploader("Upload Resume (PDF or TXT)", type=["pdf", "txt"])

if uploaded_file:
    if uploaded_file.type == "application/pdf":
        resume_text = extract_text_from_pdf(uploaded_file)
    else:
        resume_text = uploaded_file.read().decode("utf-8")

    cleaned_text = clean_text(resume_text)
    vectorized = tfidf.transform([cleaned_text])
    prediction = model.predict(vectorized)
    predicted_role = le.inverse_transform(prediction)[0]



    # Get skills for predicted role
    skills_text = " ".join(job_role_skills.get(predicted_role, []))

    # Vectorize resume & skills
    resume_vec = tfidf.transform([cleaned_text])
    skills_vec = tfidf.transform([skills_text])

    # Cosine similarity
    match_score = cosine_similarity(resume_vec, skills_vec)[0][0]
    match_percentage = round(match_score * 100, 2)


    if match_percentage >= 75:
        fit_level = "Good Match"
    elif match_percentage >= 50:
        fit_level = "Average Match"
    else:
        fit_level = "Poor Match"


    resume_words = set(cleaned_text.split())
    required_skills = job_role_skills.get(predicted_role, [])

    missing_skills = [
        skill for skill in required_skills
        if skill not in resume_words
    ][:5]


    st.success(f"✅ Predicted Job Role: **{predicted_role}**")
    st.info(f"📊 Resume–Job Match Score: **{match_percentage}%**")
    st.info(f"🎯 Fit Level: **{fit_level}**")

    if missing_skills:
        st.warning(f"⚠️ Missing Skills: {', '.join(missing_skills)}")
    else:
        st.success("🎉 All key skills are present!")

2026-01-22 08:24:31.008 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-22 08:24:31.009 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-22 08:24:31.010 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-22 08:24:31.011 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-22 08:24:31.012 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-22 08:24:31.013 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-22 08:24:31.014 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-22 08:24:31.015 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar